<a href="https://colab.research.google.com/github/Jonchyk/Datamgmt/blob/main/PS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
#---------------------------SETUP----------------------------------
#get useful libraries
import time, os, sys, re #basics
import zipfile, json, datetime, string   #string for annotating points in scatter
import numpy as np #basic math
from statistics import * #stats

!pip install pyreadstat

import matplotlib.pyplot as plt #import pylab as plt #apparently discouraged now:
 #https://stackoverflow.com/questions/11469336/what-is-the-difference-between-pylab-and-pyplot
 #https://www.tutorialspoint.com/matplotlib/matplotlib_pylab_module.htm

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler
#s4 = Styler(df4, uuid_len=0, cell_ids=False)

import urllib  #weird, guess need to have os and pandas imported for this to work  %TODO/LATER ditch it, its weird anyway, just use wget/curl

from google.colab import files

#import webbrowser

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter() #this enables spreadsheet view upon calling dataframe (without() )

#many tricks how to extend notebook functionality
#https://coderzcolumn.com/tutorials/python/list-of-useful-magic-commands-in-jupyter-notebook-lab
#will display all output not just last command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#MAGICS and THEMES/STYLES: important! does affect not just shading/colors, but also fonts, spacing, etc
#(even if you only select default (v not selecting anything) [but does seem to work better if you do make explicit sleections])

###magics: https://ipython.readthedocs.io/en/stable/interactive/magics.html
#most essential setup for vis: it does affect vis! careful!! stick with inline, maybe notebook; others mostly for non-notebook, eg spyder environ
#https://jakevdp.github.io/PythonDataScienceHandbook/04.00-introduction-to-matplotlib.html recomends *inline*!
#show current one:
#%matplotlib
#%matplotlib --list
#interactive plots:
#%matplotlib notebook
#static images of your plot:
%matplotlib inline
#may play with this one and other magics (btw default is probably agg)
#%matplotlib nbagg
##https://www.marktechpost.com/2023/10/20/6-magic-commands-for-jupyter-notebooks-in-python-data-science/
#%%latex
#%ai
#%run
#%writefile
#%history -n

###themes/styles: https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
#https://jakevdp.github.io/PythonDataScienceHandbook/04.11-settings-and-stylesheets.html
#https://matplotlib.org/stable/tutorials/introductory/customizing.html
#here more about art and style than under the hood functionality as with magics, explore and experiment
#many may find 'default' or seaborn ones more pleasing; my fav 'classic' is back from 90s ;)
#plt.style.available #list available styles :) may install more
#plt.style.use('default') # more delicate subtle than classic
plt.style.use('classic')  #  'seaborn-whitegrid' 'seaborn-white' 'seaborn-poster'
# btw: magics v theme/style sequence matters, eg if i specify classic style before inline magic, i wouldnt get grey bounding box im getting

#sometimes have to install library which you get from https://pypi.org/
#!pip install geopandas

In [93]:
#read spss file
df = pd.read_spss("fs.sav")
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns', None)  # No limit on columns
#let's take a peek at this data set.
df.head(10)
#Let's specifically look at our geographic breakdown, since this is important
df["stratum"].value_counts()

,HH1,HH2,LN,FS1,FS2,FS3,FSINT,FS4,FS5,FS6,FS7D,FS7M,FS7Y,FS9,FS10,FS17,FS8H,FS8M,FS11H,FS11M,FS12,FS13,FS14,FS15,FSHINT,FSFIN,FS15A,FS15C,FS15D,CB2M,CB2Y,CB3,CB4,CB5A,CB5B,CB6,CB7,CB8A,CB8B,CB9,CB10A,CB10B,CL1A,CL1B,CL1C,CL1X,CL3,CL4,CL5,CL6A,CL6B,CL6C,CL6D,CL6E,CL6X,CL7,CL8,CL9,CL10,CL11A,CL11B,CL11C,CL11D,CL11E,CL11F,CL11X,CL13,FCD2A,FCD2B,FCD2C,FCD2D,FCD2E,FCD2F,FCD2G,FCD2H,FCD2I,FCD2J,FCD2K,FCD3,FCD4,FCD5,FCF1,FCF2,FCF3,FCF6,FCF8,FCF10,FCF11,FCF12,FCF13,FCF14,FCF15,FCF16,FCF17,FCF18,FCF19,FCF20,FCF21,FCF22,FCF23,FCF24,FCF25,FCF26,PR3,PR5,PR6,PR7,PR8,PR9A,PR9B,PR10,PR11A,PR11B,PR12A,PR12B,PR12C,PR12X,PR13,PR15,FL1,FL2H,FL2M,FL3,FL4A,FL4B,FL4C,FL6A,FL6B,FL7,FL9,FL10,FL10C,FL13,FL14,FL15,FL16,FL17,FL18,FL19INTRO,FL19W1,FL19W2,FL19W3,FL19W4,FL19W5,FL19W6,FL19W7,FL19W8,FL19W9,FL19W10,FL19W11,FL19W12,FL19W13,FL19W14,FL19W15,FL19W16,FL19W17,FL19W18,FL19W19,FL19W20,FL19W21,FL19W22,FL19W23,FL19W24,FL19W25,FL19W26,FL19W27,FL19W28,FL19W29,FL19W30,FL19W31,FL19W32,FL19W33,FL19W34,FL19W35,FL19W36,FL19W37,FL19W38,FL19W39,FL19W40,FL19W41,FL19W42,FL19W43,FL19W44,FL19W45,FL19W46,FL19W47,FL19W48,FL19W49,FL19W50,FL19W51,FL19W52,FL19W53,FL19W54,FL19W55,FL19W56,FL19W57,FL19W58,FL19W59,FL19W60,FL19W61,FL19W62,FL19W63,FL19W64,FL19W65,FL19W66,FL19W67,FL19W68,FL19W69,FL19W70,FL19W71,FL19W72,FL19END,FL20A,FL20B,FL21BA,FL21BB,FL21BC,FL21BD,FL21BE,FL21D,FL21G,FL21H,FL21I,FL21J,FL21K,FL21L,FL21OINTRO,FL21OW1,FL21OW2,FL21OW3,FL21OW4,FL21OW5,FL21OW6,FL21OW7,FL21OW8,FL21OW9,FL21OW10,FL21OW11,FL21OW12,FL21OW13,FL21OW14,FL21OW15,FL21OW16,FL21OW17,FL21OW18,FL21OW19,FL21OW20,FL21OW21,FL21OW22,FL21OW23,FL21OW24,FL21OW25,FL21OW26,FL21OW27,FL21OW28,FL21OW29,FL21OW30,FL21OW31,FL21OW32,FL21OW33,FL21OW34,FL21OW35,FL21OW36,FL21OW37,FL21OW38,FL21OW39,FL21OW40,FL21OW41,FL21OW42,FL21OW43,FL21OW44,FL21OW45,FL21OW46,FL21OW47,FL21OW48,FL21OW49,FL21OW50,FL21OW51,FL21OW52,FL21OW53,FL21OW54,FL21OEND,FL21PA,FL21PB,FL22A,FL22B,FL22C,FL22D,FL22E,FL23A,FL23B,FL23C,FL23D,FL23E,FL23F,FL24A,FL24B,FL24C,FL24D,FL24E,FL25A,FL25B,FL25C,FL25D,FL25E,FL26,FL26A,FL26B,FL26C,FL26D,FL26E,FL27A,FL27B,FL27C,FL27D,FL27E,FL28,ST1LANG,ST1WNUM,ST2LANG,ST2WNUM,HH4,HH6,HH7,HH52,HL4,ED5A,ED5B,FSAGE,FSDOI,FSDOB,fshweight,fsweight,melevel,fselevel,schage,fsdisability,ethnicity,caretakerdis,wscore,windex5,windex10,wscoreu,windex5u,windex10u,wscorer,windex5r,windex10r,PSU,stratum
0,1.0,8.0,3.0,1.0,8.0,3.0,35.0,2.0,35.0,30.0,22.0,11.0,2023.0,"NO, FIRST INTERVIEW",YES,COMPLETED,17.0,30.0,17.0,53.0,KYRGYZ,KYRGYZ,KYRGYZ,"NO, NOT USED",35.0,SAVE QUESTIONNAIRE AND FINISH,NO,NO,NaN,MAY,2011.0,12.0,YES,BASIC SECONDARY,GRADE 6,NO,YES,BASIC SECONDARY,GRADE 6,YES,BASIC SECONDARY,GRADE 5,NO,NO,NO,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NaN,NO,NaN,YES,NO,YES,NO,NO,NO,YES,5.0,YES,YES,NO,YES,YES,NO,NO,NO,NO,NO,NO,NO,NaN,NO,NO,NO,NO,NO DIFFICULTY,NO DIFFICULTY,NaN,NaN,NaN,NaN,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,A FEW TIMES A YEAR,NEVER,TEN OR MORE BOOKS,YES,NO,YES,YES,YES,NO,YES,YES,NO,NO,NO,NO,NO,NO,NaN,"YES, PERMISSION IS GIVEN",17.0,30.0,YES,You are not alone with the child unless he/she...,You have engaged the child in conversation and...,"The child is sat comfortably, able to use the ...",YES,NO,RUSSIAN,RUSSIAN,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONTINUE,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,NOT REACHED,CONTINUE,71.0,0.0,CORRECT(TWO),CORRECT(FLOWERS),CORRECT(BECAUSE HE FELL),CORRE

,count
stratum,
Naryn - rural,480
Talas - rural,443
Osh city - urban,430
Osh - rural,387
Chui - rural,360
Batken - rural,327
Bishkek city - urban,308
Jala-Abad - rural,296
Issyk-kul - rural,284


In [114]:
#let's get a full list of columns
print(list(df.columns))

['FS10', 'FS12', 'FS13', 'FS14', 'CB3', 'CB4', 'CB5B', 'CL1A', 'CL1B', 'CL1C', 'CL1X', 'CL3', 'PR3', 'PR5', 'PR6', 'PR12A', 'PR12B', 'PR12C', 'PR12X', 'FL3', 'FL6A', 'FL6B', 'FL7', 'FL9', 'FL10C', 'FL13', 'FL14', 'FL15', 'FL16', 'FL17', 'FL18', 'FL19INTRO', 'FL19W1', 'FL19W2', 'FL19W3', 'FL19W4', 'FL19W5', 'FL19W6', 'FL19W7', 'FL19W8', 'FL19W9', 'FL19W10', 'FL19W11', 'FL19W12', 'FL19W13', 'FL19W14', 'FL19W15', 'FL19W16', 'FL19W17', 'FL19W18', 'FL19W19', 'FL19W20', 'FL19W21', 'FL19W22', 'FL19W23', 'FL19W24', 'FL19W25', 'FL19W26', 'FL19W27', 'FL19W28', 'FL19W29', 'FL19W30', 'FL19W31', 'FL19W32', 'FL19W33', 'FL19W34', 'FL19W35', 'FL19W36', 'FL19W37', 'FL19W38', 'FL19W39', 'FL19W40', 'FL19W41', 'FL19W42', 'FL19W43', 'FL19W44', 'FL19W45', 'FL19W46', 'FL19W47', 'FL19W48', 'FL19W49', 'FL19W50', 'FL19W51', 'FL19W52', 'FL19W53', 'FL19W54', 'FL19W55', 'FL19W56', 'FL19W57', 'FL19W58', 'FL19W59', 'FL19W60', 'FL19W61', 'FL19W62', 'FL19W63', 'FL19W64', 'FL19W65', 'FL19W66', 'FL19W67', 'FL19W68', 'FL

In [115]:
#ok. Now that i've done this. I've identified all of the "columns" that need to be kept. it's a lot. Thankfully, we are keeping all of the FL codes (foundational learning)
#Let's drop all non necessary columns here, keeping language/geographic/ethnicity, grade, gender, age, questions on child labor, natural disasters, and the full foundational assessment questionaire
df = df.loc[:, df.columns.isin(['FS10', 'FS12', 'FS13', 'FS14', 'CB3', 'CB4', 'CB5B', 'CL1A' , 'CL1B', 'CL1C', 'CL1X', 'CL3', 'PR3', 'PR5', 'PR6', 'PR12A', 'PR12B', 'PR12C', 'PR12X', 'FL3', 'FL6A', 'FL6B', 'FL7', 'FL9', 'FL10C', 'FL13', 'FL14', 'FL15', 'FL16', 'FL17', 'FL18', 'FL19INTRO', 'FL19W1', 'FL19W2', 'FL19W3', 'FL19W4', 'FL19W5', 'FL19W6', 'FL19W7', 'FL19W8', 'FL19W9', 'FL19W10', 'FL19W11', 'FL19W12', 'FL19W13', 'FL19W14', 'FL19W15', 'FL19W16', 'FL19W17', 'FL19W18', 'FL19W19', 'FL19W20', 'FL19W21', 'FL19W22', 'FL19W23', 'FL19W24', 'FL19W25', 'FL19W26', 'FL19W27', 'FL19W28', 'FL19W29', 'FL19W30', 'FL19W31', 'FL19W32', 'FL19W33', 'FL19W34', 'FL19W35', 'FL19W36', 'FL19W37', 'FL19W38', 'FL19W39', 'FL19W40', 'FL19W41', 'FL19W42', 'FL19W43', 'FL19W44', 'FL19W45', 'FL19W46', 'FL19W47', 'FL19W48', 'FL19W49', 'FL19W50', 'FL19W51', 'FL19W52', 'FL19W53', 'FL19W54', 'FL19W55', 'FL19W56', 'FL19W57', 'FL19W58', 'FL19W59', 'FL19W60', 'FL19W61', 'FL19W62', 'FL19W63', 'FL19W64', 'FL19W65', 'FL19W66', 'FL19W67', 'FL19W68', 'FL19W69', 'FL19W70', 'FL19W71', 'FL19W72', 'FL19END', 'FL20A', 'FL20B', 'FL21BA', 'FL21BB', 'FL21BC', 'FL21BD', 'FL21BE', 'FL21D', 'FL21H', 'FL21I', 'FL21J', 'FL21K', 'FL21L', 'FL21OINTRO', 'FL21OW1', 'FL21OW2', 'FL21OW3', 'FL21OW4', 'FL21OW5', 'FL21OW6', 'FL21OW7', 'FL21OW8', 'FL21OW9', 'FL21OW10', 'FL21OW11', 'FL21OW12', 'FL21OW13', 'FL21OW14', 'FL21OW15', 'FL21OW16', 'FL21OW17', 'FL21OW18', 'FL21OW19', 'FL21OW20', 'FL21OW21', 'FL21OW22', 'FL21OW23', 'FL21OW24', 'FL21OW25', 'FL21OW26', 'FL21OW27', 'FL21OW28', 'FL21OW29', 'FL21OW30', 'FL21OW31', 'FL21OW32', 'FL21OW33', 'FL21OW34', 'FL21OW35', 'FL21OW36', 'FL21OW37', 'FL21OW38', 'FL21OW39', 'FL21OW40', 'FL21OW41', 'FL21OW42', 'FL21OW43', 'FL21OW44', 'FL21OW45', 'FL21OW46', 'FL21OW47', 'FL21OW48', 'FL21OW49', 'FL21OW50', 'FL21OW51', 'FL21OW52', 'FL21OW53', 'FL21OW54', 'FL21OEND', 'FL21PA', 'FL21PB', 'FL22A', 'FL22B', 'FL22C', 'FL22D', 'FL22E', 'FL23A', 'FL23B', 'FL23C', 'FL23D', 'FL23E', 'FL23F', 'FL24A', 'FL24B', 'FL24C', 'FL24D', 'FL24E', 'FL25A', 'FL25B', 'FL25C', 'FL25D', 'FL25E', 'FL26', 'FL26A', 'FL26B', 'FL26C', 'FL26D', 'FL26E', 'FL27A', 'FL27B', 'FL27C', 'FL27D', 'FL27E', 'FL28', 'ST1LANG', 'ST1WNUM', 'ST2LANG', 'ST2WNUM', 'HH4', 'HH6', 'HH7', 'HH52', 'HL4', 'ED5A', 'ED5B', 'FSAGE', 'fsweight', 'fselevel', 'ethnicity','wscore', 'windex5', 'windex10', 'stratum'])]

In [96]:
#let's get this down to 7 to 10 years old, since the data collected is for ages 7-14, but is a G2 foundational test. Wow!
df_age = df[(df["CB3"] > 6) & (df["CB3"] < 11)]
#how many respondents do we have now?
df_age["stratum"].value_counts()

,count
stratum,
Naryn - rural,152
Osh city - urban,143
Osh - rural,138
Talas - rural,132
Chui - rural,128
Batken - rural,106
Issyk-kul - rural,100
Bishkek city - urban,98
Jala-Abad - rural,97


In [97]:
#further, let's see how many of these children consented to taking the test.
df_age["FL3"].value_counts()

,count
FL3,
YES,1219
NO / NOT ASKED,48


In [98]:
#1219 said yes! Of those 1219, how many do we have in each "stratum" now?
df_age[["stratum", "FL3"]].value_counts(normalize=False)
#ok! This looks like it's enough data to work with.  While some of the urban smaples are a bit low, every sample per region is >30!


,,count
stratum,FL3,
Naryn - rural,YES,149
Osh city - urban,YES,130
Talas - rural,YES,128
Chui - rural,YES,124
Osh - rural,YES,114
Batken - rural,YES,100
Issyk-kul - rural,YES,100
Bishkek city - urban,YES,80
Jala-Abad - rural,YES,78


In [100]:
#run the dataframe and double check that we have only the columns listed above and all looks ok.
df_age

,FS10,FS12,FS13,FS14,CB3,CB4,CB5B,CL1A,CL1B,CL1C,CL1X,CL3,PR3,PR5,PR6,PR12A,PR12B,PR12C,PR12X,FL3,FL6A,FL6B,FL7,FL9,FL10C,FL13,FL14,FL15,FL16,FL17,FL18,FL19INTRO,FL19W1,FL19W2,FL19W3,FL19W4,FL19W5,FL19W6,FL19W7,FL19W8,FL19W9,FL19W10,FL19W11,FL19W12,FL19W13,FL19W14,FL19W15,FL19W16,FL19W17,FL19W18,FL19W19,FL19W20,FL19W21,FL19W22,FL19W23,FL19W24,FL19W25,FL19W26,FL19W27,FL19W28,FL19W29,FL19W30,FL19W31,FL19W32,FL19W33,FL19W34,FL19W35,FL19W36,FL19W37,FL19W38,FL19W39,FL19W40,FL19W41,FL19W42,FL19W43,FL19W44,FL19W45,FL19W46,FL19W47,FL19W48,FL19W49,FL19W50,FL19W51,FL19W52,FL19W53,FL19W54,FL19W55,FL19W56,FL19W57,FL19W58,FL19W59,FL19W60,FL19W61,FL19W62,FL19W63,FL19W64,FL19W65,FL19W66,FL19W67,FL19W68,FL19W69,FL19W70,FL19W71,FL19W72,FL19END,FL20A,FL20B,FL21BA,FL21BB,FL21BC,FL21BD,FL21BE,FL21D,FL21H,FL21I,FL21J,FL21K,FL21L,FL21OINTRO,FL21OW1,FL21OW2,FL21OW3,FL21OW4,FL21OW5,FL21OW6,FL21OW7,FL21OW8,FL21OW9,FL21OW10,FL21OW11,FL21OW12,FL21OW13,FL21OW14,FL21OW15,FL21OW16,FL21OW17,FL21OW18,FL21OW19,FL21OW20,FL21OW21,FL21OW22,FL21OW23,FL21OW24,FL21OW25,FL21OW26,FL21OW27,FL21OW28,FL21OW29,FL21OW30,FL21OW31,FL21OW32,FL21OW33,FL21OW34,FL21OW35,FL21OW36,FL21OW37,FL21OW38,FL21OW39,FL21OW40,FL21OW41,FL21OW42,FL21OW43,FL21OW44,FL21OW45,FL21OW46,FL21OW47,FL21OW48,FL21OW49,FL21OW50,FL21OW51,FL21OW52,FL21OW53,FL21OW54,FL21OEND,FL21PA,FL21PB,FL22A,FL22B,FL22C,FL22D,FL22E,FL23A,FL23B,FL23C,FL23D,FL23E,FL23F,FL24A,FL24B,FL24C,FL24D,FL24E,FL25A,FL25B,FL25C,FL25D,FL25E,FL26,FL26A,FL26B,FL26C,FL26D,FL26E,FL27A,FL27B,FL27C,FL27D,FL27E,FL28,ST1LANG,ST1WNUM,ST2LANG,ST2WNUM,HH4,HH6,HH7,HH52,HL4,ED5A,ED5B,FSAGE,fsweight,fselevel,ethnicity,wscore,windex5,windex10,stratum
2,YES,KYRGYZ,KYRGYZ,KYRGYZ,7.0,YES,GRADE 1,NO,NO,NO,NO,NaN,NINE,YES,YES,NO,NO,NO,NO,YES,YES,YES,KYRGYZ,RUSSIAN,NaN,CONTINUE,YES,CORRECT (5),NaN,CORRECT (ALABAI),NaN,CONTINUE,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,INCORRECT,CORRECT,INCORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,INCORRECT,CORRECT,CORRECT,INCORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,NOT REACHED,CONTINUE,71.0,4.0,CORRECT(TWO),CORRECT(FLOWERS),CORRECT(BECAUSE HE FELL),CORRECT(NEAR A KARAGACH TREE),CORRECT (BECAUSE THE FARMER GAVE HIM MANY FLOW...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CORRECT(9),CORRECT( 12 ),CORRECT( 30 ),CORRECT( 48 ),CORRECT( 74 ),CORRECT( 731 ),CORRECT (7),CORRECT (24),CORRECT (58),CORRECT (67),CORRECT (154),CORRECT (5),CORRECT (14),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,RUSSIAN,71.0,WAS NOT NECESSARY TO OFFER,NaN,30.0,URBAN,Issyk-kul,3.0,FEMALE,HIGHER,GRADE 4,5-9,1.825544,Pre-school or none/Primary,Kyrgyz,0.390934,Fourth,8th decile,Issyk-kul - urban
8,YES,KYRGYZ,KYRGYZ,KYRGYZ,7.0,YES,GRADE 2,NO,NO,NO,NO,NaN,TWO,YES,YES,NO,NO,NO,NO,YES,YES,YES,KYRGYZ,RUSSIAN,NaN,CONTINUE,YES,CORRECT (5),NaN,CORRECT (ALABAI),NaN,CONTINUE,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,INCORRECT,CORR

In [116]:
#There are two stories. The first story is read in a language selected by the child. If the child agrees to read a second story, in FL21D, they must pick a second language to read it in if they consent. Let's see that break out
pd.crosstab(df_age['FL21D'], df_age['ethnicity'])


ethnicity,Kyrgyz,Other ethnicity,Russian,Uzbek
FL21D,,,,
DOES NOT WANT TO TRY,228,7,10,47
KYRGYZ,89,5,2,13
NO RESPONSE,31,1,0,1
RUSSIAN,138,1,3,8
UZBEK,2,0,0,19


In [117]:
#Another question remains; the questionaire asks children what language they speak at home, versus hear at school (FL7, FL9). How does those counts come about?
pd.crosstab(df_age['FL7'], df_age['FL9'])
#by basis of ethnicity compared to language of instruction
pd.crosstab(df_age['ethnicity'], df_age['FL9'])
#by basis of ethnicity compared to language at home
pd.crosstab(df_age['ethnicity'], df_age['FL7'])



FL9,KYRGYZ,NO RESPONSE,RUSSIAN,TAJIK,UZBEK
FL7,,,,,
15.0,0,0,1,2,0
16.0,0,0,3,1,0
17.0,0,0,1,0,0
KYRGYZ,730,0,198,0,2
NO RESPONSE,0,1,0,0,0
OTHER,1,0,1,0,0
RUSSIAN,5,0,139,0,0
TAJIK,0,0,0,1,0
UZBEK,13,0,67,0,50


FL9,KYRGYZ,NO RESPONSE,RUSSIAN,TAJIK,UZBEK
ethnicity,,,,,
Kyrgyz,726,0,259,0,3
Other ethnicity,9,0,26,4,1
Russian,0,0,37,0,0
Uzbek,14,1,88,0,48


FL7,15.0,16.0,17.0,KYRGYZ,NO RESPONSE,OTHER,RUSSIAN,TAJIK,UZBEK
ethnicity,,,,,,,,,
Kyrgyz,0,0,0,912,0,0,75,0,3
Other ethnicity,3,4,1,10,0,2,16,1,4
Russian,0,0,0,0,0,0,37,0,0
Uzbek,0,0,0,9,1,0,17,0,124


In [120]:
#Let's rename certain columns, not all of them, but a few critical pieces
df_age.rename(columns={'FS10': 'consent', 'FS17': 'completed', 'FS12': 'language_questionaire', 'FS13': 'language_interview', 'FS14': 'native_language', 'CB5': 'highest_grade'}, inplace=False)

In [121]:
#check columns rename went through, use false first, then rerun below with true
df_age.head(1)

,consent,language_questionaire,language_interview,native_language,CB3,CB4,CB5B,CL1A,CL1B,CL1C,CL1X,CL3,PR3,PR5,PR6,PR12A,PR12B,PR12C,PR12X,FL3,FL6A,FL6B,FL7,FL9,FL10C,FL13,FL14,FL15,FL16,FL17,FL18,FL19INTRO,FL19W1,FL19W2,FL19W3,FL19W4,FL19W5,FL19W6,FL19W7,FL19W8,FL19W9,FL19W10,FL19W11,FL19W12,FL19W13,FL19W14,FL19W15,FL19W16,FL19W17,FL19W18,FL19W19,FL19W20,FL19W21,FL19W22,FL19W23,FL19W24,FL19W25,FL19W26,FL19W27,FL19W28,FL19W29,FL19W30,FL19W31,FL19W32,FL19W33,FL19W34,FL19W35,FL19W36,FL19W37,FL19W38,FL19W39,FL19W40,FL19W41,FL19W42,FL19W43,FL19W44,FL19W45,FL19W46,FL19W47,FL19W48,FL19W49,FL19W50,FL19W51,FL19W52,FL19W53,FL19W54,FL19W55,FL19W56,FL19W57,FL19W58,FL19W59,FL19W60,FL19W61,FL19W62,FL19W63,FL19W64,FL19W65,FL19W66,FL19W67,FL19W68,FL19W69,FL19W70,FL19W71,FL19W72,FL19END,FL20A,FL20B,FL21BA,FL21BB,FL21BC,FL21BD,FL21BE,FL21D,FL21H,FL21I,FL21J,FL21K,FL21L,FL21OINTRO,FL21OW1,FL21OW2,FL21OW3,FL21OW4,FL21OW5,FL21OW6,FL21OW7,FL21OW8,FL21OW9,FL21OW10,FL21OW11,FL21OW12,FL21OW13,FL21OW14,FL21OW15,FL21OW16,FL21OW17,FL21OW18,FL21OW19,FL21OW20,FL21OW21,FL21OW22,FL21OW23,FL21OW24,FL21OW25,FL21OW26,FL21OW27,FL21OW28,FL21OW29,FL21OW30,FL21OW31,FL21OW32,FL21OW33,FL21OW34,FL21OW35,FL21OW36,FL21OW37,FL21OW38,FL21OW39,FL21OW40,FL21OW41,FL21OW42,FL21OW43,FL21OW44,FL21OW45,FL21OW46,FL21OW47,FL21OW48,FL21OW49,FL21OW50,FL21OW51,FL21OW52,FL21OW53,FL21OW54,FL21OEND,FL21PA,FL21PB,FL22A,FL22B,FL22C,FL22D,FL22E,FL23A,FL23B,FL23C,FL23D,FL23E,FL23F,FL24A,FL24B,FL24C,FL24D,FL24E,FL25A,FL25B,FL25C,FL25D,FL25E,FL26,FL26A,FL26B,FL26C,FL26D,FL26E,FL27A,FL27B,FL27C,FL27D,FL27E,FL28,ST1LANG,ST1WNUM,ST2LANG,ST2WNUM,HH4,HH6,HH7,HH52,HL4,ED5A,ED5B,FSAGE,fsweight,fselevel,ethnicity,wscore,windex5,windex10,stratum
2,YES,KYRGYZ,KYRGYZ,KYRGYZ,7.0,YES,GRADE 1,NO,NO,NO,NO,NaN,NINE,YES,YES,NO,NO,NO,NO,YES,YES,YES,KYRGYZ,RUSSIAN,NaN,CONTINUE,YES,CORRECT (5),NaN,CORRECT (ALABAI),NaN,CONTINUE,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,INCORRECT,CORRECT,INCORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,INCORRECT,CORRECT,CORRECT,INCORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,NOT REACHED,CONTINUE,71.0,4.0,CORRECT(TWO),CORRECT(FLOWERS),CORRECT(BECAUSE HE FELL),CORRECT(NEAR A KARAGACH TREE),CORRECT (BECAUSE THE FARMER GAVE HIM MANY FLOW...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CORRECT(9),CORRECT( 12 ),CORRECT( 30 ),CORRECT( 48 ),CORRECT( 74 ),CORRECT( 731 ),CORRECT (7),CORRECT (24),CORRECT (58),CORRECT (67),CORRECT (154),CORRECT (5),CORRECT (14),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,RUSSIAN,71.0,WAS NOT NECESSARY TO OFFER,NaN,30.0,URBAN,Issyk-kul,3.0,FEMALE,HIGHER,GRADE 4,5-9,1.825544,Pre-school or none/Primary,Kyrgyz,0.390934,Fourth,8th decile,Issyk-kul - urban


In [122]:
#repeat df rename function but htis time with inplace = true

In [105]:
#NEXT STEPS for this Dataset:
#ok, this is interesting. I'll need to look into this deeper when thinking about how to compare and contrast. Did students read in their language of instruction, or language at home?
#For the next PS, will's double check and create a column that returns 'languageathome1" or "languageofinstruction1", if the child chose to read a storyu, was it based on language at home or language of instruction?
#Will likely drop the second story. We'll assess foundational knowledge based on percentage of reading and math.
#Then run the average score in each of those  categories, based on the stratum (Oblast, rural/urban)
#then compare results by age, and by gender


In [113]:
# In the future can do a merge and roll up, for now, let's keep as is. Need to export this to a CSV file, right?

In [124]:
#now to read in a second dataset - let's grab some data from UNGLOBAL Data lab by region, it's quite clean and ready to go!
pop = pd.read_csv('/content/GDL-Total-area-population-in-millions-data.csv')
pop.head(1)

,Country,Continent,ISO_Code,Level,GDLCODE,Region,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kyrgyz Republic,Asia/Pacific,KGZ,National,KGZt,Total,NaN,NaN,4.52,4.58,4.64,4.7,4.75,4.81,4.87,4.93,4.99,5.04,5.1,5.16,5.22,5.28,5.35,5.41,5.48,5.54,5.61,5.72,5.84,5.96,6.08,6.2,6.32,6.44,6.57,6.69,NaN,NaN


In [ ]:
pop.drop(columns=['Country', 'ISO_Code', 'Level', 'GDLCODE'], inplace=False)
pop.head(10)

In [ ]:
#now let's get a third data set, this is all clean and ready to go from the Kyrgyzstan National Stats Bureau